In [1]:
# References 
#https://github.com/mlfoundations/open_clip?tab=readme-ov-file

In [2]:
import numpy as np
import torch
import warnings
import shutil
import open_clip
from open_clip import tokenizer
import time

import os, glob
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

from collections import OrderedDict
import torch

import importlib
import CLIP_Module
import CLIP_GA
importlib.reload(CLIP_Module)
importlib.reload(CLIP_GA)

from CLIP_Module import *  # Re-import function to reflect updates
from CLIP_GA import *



%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Ignore all warnings5
warnings.filterwarnings("ignore")

In [3]:
# Check if the file exists before reading
file_path = "Florida_Images_Folders02202025.txt"

if os.path.exists(file_path):
    with open(file_path, "r") as f:
        Florida_Images_Folders = [line.strip() for line in f]
    print("File loaded successfully.")
    
else:
    print(f"File '{file_path}' does not exist.")
    Florida_Images_Folders1 = glob.glob("/blue/changzhao/GeoAI-CES/Data/unzipdata/*/*.jpg") #entire US 2016~2021
    Florida_Images_Folders2 = glob.glob("/blue/changzhao/GeoAI-CES/Data/Raw_Flicker_Metadata/Flickr_US_2013_2016_Images/*/*.jpg") #entire US 2013~2016
    Florida_Images_Folders3 = glob.glob("/blue/changzhao/haoyuliao/CES/LabelState&DowoloadMissImgs/Missing_Images/*/*.jpg") #entire US 2012~2021
    Florida_Images_Folders4 = glob.glob("/blue/changzhao/haoyuliao/CES/LabelState&DowoloadMissImgs/HuntingDownload/*.jpg") #entire US 2012~2021
    Florida_Images_Folders5 = glob.glob("/blue/changzhao/haoyuliao/CES/LabelState&DowoloadMissImgs/IncreaseImgFromD5/*/*.jpg") #entire US 2012~2021

    Florida_Images_Folders = Florida_Images_Folders1.copy()  # To avoid modifying the original list1
    Florida_Images_Folders.extend(Florida_Images_Folders2)
    Florida_Images_Folders.extend(Florida_Images_Folders3)
    Florida_Images_Folders.extend(Florida_Images_Folders4)
    Florida_Images_Folders.extend(Florida_Images_Folders5)

    # Save the complete list to a text file
    with open("Florida_Images_Folders02202025.txt", "w") as f:
        for item in Florida_Images_Folders:
            f.write(f"{item}\n")

File loaded successfully.


In [4]:
images_paths_map = {x.split('/')[-1].split('_')[0]: x for x in Florida_Images_Folders}

In [5]:
metadata = pd.read_csv('flickr_all_FL_201312_202112_RmMissPID_UrbanArea_SatePark_NationalPark_ParkServer_ProtecedArea_part3.csv')

In [6]:
layer5_drop = metadata.drop_duplicates(subset='photoid')
print(f'Before {len(metadata)}, After {len(layer5_drop)}')

Before 224902, After 224902


In [7]:
metadata['photoid'] = metadata['photoid'].astype(str)

In [8]:
metadata.reset_index(drop=True, inplace=True)
metadata = metadata[metadata['photoid'].isin(images_paths_map)]
metadata.reset_index(drop=True, inplace=True)
len(metadata)

224902

In [9]:
photo_ids = [str(x) for x in metadata['photoid']]

In [10]:
import psutil

# Get the memory usage in bytes
memory_usage = psutil.Process().memory_info().rss

# Convert to megabytes
memory_usage_mb = memory_usage / (1024 ** 2)
print(f"Memory Usage: {memory_usage_mb:.2f} MB")

Memory Usage: 10118.64 MB


In [11]:
# Load the object from the file
import pickle
'''
with open('images_paths_map_US-2013~2021.pkl', 'wb') as file:
    pickle.dump(images_paths_map, file)

with open('images_paths_map_US-2013~2021.pkl', 'rb') as file:
    loaded_obj = pickle.load(file)

print("Object loaded successfully.")
print(loaded_obj)
'''

'\nwith open(\'images_paths_map_US-2013~2021.pkl\', \'wb\') as file:\n    pickle.dump(images_paths_map, file)\n\nwith open(\'images_paths_map_US-2013~2021.pkl\', \'rb\') as file:\n    loaded_obj = pickle.load(file)\n\nprint("Object loaded successfully.")\nprint(loaded_obj)\n'

In [12]:
# Directory to save results
def nameCover(Pre_name):
    if Pre_name == "Hiking":
        return "Hiking"
    
    if Pre_name == "Backpacking":
        return "Backpacking"
        #return "Hiking"
    
    if Pre_name == "Boating":
        return "Boating"    
    
    if Pre_name == "Swimming":
        return "Swimming"
    
    if Pre_name == "Camping":
        return "Camping"
    
    if Pre_name == "Fishing":
        return "Fishing"
    
    if Pre_name == "Biking":
        return "Biking"
    
    if Pre_name == "Horseback_Riding":
        return "Horseback_Riding"
    
    if Pre_name == "Wildlife_Viewing":
        return "Wildlife_Viewing"
     
    if Pre_name == "Shelling":
        return "Shelling" 
    
    if Pre_name == "Surfing":
        return "Surfing"
    
    if Pre_name == "Hunting":
        return "Hunting"
    
    if Pre_name == "Landscape_Aesthetics":
        return "Landscape_Aesthetics"    
    
    if Pre_name == "No_CES":
        return "No_CES"   


In [13]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_image_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_text_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_basic_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_s13m_b4k'),
 ('ViT-

In [14]:
def GetPred(similarity):
    # Find the index of the highest similarity score for each image
    predicted_indices = np.argmax(similarity, axis=0)

    ## Use these indices to get the corresponding labels
    predicted_labels = [class_names[idx] for idx in predicted_indices]

    predicted_cover_labels = [nameCover(x) for x in predicted_labels]

    # Add a new column 'pred_res' with default values (e.g., None or any initial value you prefer)
    #metadata['Clip_Predictions'] = predicted_cover_labels ##
    
    top_indices_per_column = np.argsort(similarity, axis=0)[::-1]

    # Get the top 5 values in each column
    top_values_per_column = np.take_along_axis(similarity, top_indices_per_column, axis=0)

    # Create a new matrix with the top 5 class names for each column
    top_class_names_matrix = np.empty_like(top_indices_per_column, dtype=object)

    for i in range(top_indices_per_column.shape[0]):
        for j in range(top_indices_per_column.shape[1]):
            top_class_names_matrix[i, j] = nameCover(class_names[top_indices_per_column[i, j]])
            
    tops = np.hstack((np.transpose(top_class_names_matrix),np.transpose(top_values_per_column)))
    
    '''
    print("Top 5 indices per column:")
    print(top_5_indices_per_column)

    print("Top 5 values per column:")
    print(top_5_values_per_column)

    # Print the matrix
    print("Top 5 class names matrix:")
    print(top_5_class_names_matrix)
    '''
    
    return predicted_cover_labels, tops

In [15]:
# Load CLIP Model
model_name = 'MobileCLIP-S1'  # Replace with your model
pretrained = 'datacompdr'      # Specify the pretrained source
model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
tokenizer = open_clip.get_tokenizer(model_name)  # Correct tokenizer initialization

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval() ##Important to turn on.


CustomTextCLIP(
  (visual): TimmModel(
    (trunk): FastVit(
      (stem): Sequential(
        (0): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList(
            (0): ConvNormAct(
              (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (bn): BatchNormAct2d(
                64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
                (drop): Identity()
                (act): Identity()
              )
            )
          )
          (conv_scale): ConvNormAct(
            (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
            (bn): BatchNormAct2d(
              64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Identity()
            )
          )
          (act): GELU(approximate='none')
        )
        (1): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList

In [16]:
import CLIP_Module
import CLIP_GA
importlib.reload(CLIP_Module)
importlib.reload(CLIP_GA)

from CLIP_Module import *  # Re-import function to reflect updates
from CLIP_GA import *


#save_dir = f'CLIP-important-promptV3/{checkPrompt}'
dn = "ALL_FL"
save_dir = f'CLIP-GPU-All_FLv4-04072025-part3' ###
shutil.rmtree(save_dir, ignore_errors=True)
os.makedirs(f'{save_dir}/PredRes/', exist_ok=True)

descriptions = {
    "Hiking": "This is an image of with people, showning hiking with a casual walk on challenging scenic trails or breathtaking scenic countryside.", #"This is an image of people walking with hiking poles, walking sticks, and wearing backpacking.",
    "Boating": "This is boating such as powerboats or man-powered vessels (rowing or paddle boats).", #su1-cb4
    "Swimming":"This is an image of people wearing swimsuits while swimming.",
    "Camping":"This is camping.",
    "Fishing":"This is fishing.",
    "Biking":"This is biking.",
    "Horseback_Riding":"This is horseback riding.",
    "Wildlife_Viewing":"This is an image without people showing wildlife viewing such as birds, flowers, or fungi.", #w3
    "Shelling":"This is shelling.",
    "Surfing": "This is an image of people surfing or holding a board.", #Su1
    "Hunting": "A hunter poses with a large kill, gutting the meat of feral animals.", #cb2-H10
    "Landscape_Aesthetics":"This is an image without people, showing the natural soothing vibrant panorama site." #cb2 97% for LA
}


class_names = ["Hiking", "Boating", "Swimming", "Camping", "Fishing", "Biking", "Horseback_Riding", "Wildlife_Viewing",
              "Shelling", "Surfing", "Hunting","Landscape_Aesthetics"]

texts = []
for name in descriptions:
    texts.append(descriptions[name])

     
text_tokens = tokenizer([desc for desc in texts]).to(device)

#similarityCol = []
images = []
images_valid = [] #1 valid; 0: corruption
original_images = []
predicted_cover_labels_col = []
#images.append(preprocess(image).to(device))
#image_input = torch.stack(images).to(device)
# Start the timer
start_time = time.perf_counter()
n = 1
for idx in range(len(photo_ids)):
    sub_start_time = time.perf_counter()
    #image_class = metadata['label'].iloc[idx]
    try:
        image_path = images_paths_map[str(photo_ids[idx])]
        image = Image.open(image_path).convert("RGB")
        images_valid.append(1)
    except:
        image_path = images_paths_map[str(photo_ids[0])] #Fake Image
        image = Image.open(image_path).convert("RGB")
        images_valid.append(0)
        continue
    
    #original_images.append(image)
    #images.append(preprocess(image).to(device)) #@@@
    #imgPro = preprocess(image).to(device)
    #imgPro = imgPro.unsqueeze(0)
    images.append(preprocess(image).to(device))
    
    if (idx+1)%500 == 0 or (idx+1) == len(photo_ids):
        image_input = torch.stack(images).to(device)
        with torch.no_grad():
            #image_features = model.encode_image(image_subinput[i]).float()
            image_features = model.encode_image(image_input).float()
            text_features = model.encode_text(text_tokens).float()

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        subsimilarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
        predicted_cover_labels, top = GetPred(subsimilarity)
        predicted_cover_labels_col += predicted_cover_labels
        #np.save(os.path.join(save_dir,f'{n}_top.npy'), top)
        
        if n == 1:
            tops = top.copy()
        else:
            tops = np.vstack((tops, top.copy()))
        
        n+=1
    
        #similarityCol.append(subsimilarity)
        #print(subsimilarity)
        del image_input, image_features, text_features, subsimilarity, images, predicted_cover_labels, top
        torch.cuda.empty_cache()  # Clear unused memory
        images = []
        
        # Get the memory usage in bytes
        memory_usage = psutil.Process().memory_info().rss

        # Convert to megabytes
        memory_usage_mb = memory_usage / (1024 ** 2)
        
        # End the timer
        sub_end_time = time.perf_counter()
        # Calculate the time taken
        sub_execution_time = sub_end_time - sub_start_time
        print(f"{n} Execution time: {sub_execution_time:.6f} seconds")
        print(f"Memory Usage: {memory_usage_mb:.2f} MB")
    
# End the timer
end_time = time.perf_counter()
# Calculate the time taken
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")

2 Execution time: 303.932252 seconds
Memory Usage: 10505.33 MB
3 Execution time: 0.638820 seconds
Memory Usage: 10551.74 MB
4 Execution time: 0.648506 seconds
Memory Usage: 10551.74 MB
5 Execution time: 0.677468 seconds
Memory Usage: 10552.00 MB
6 Execution time: 0.648965 seconds
Memory Usage: 10552.25 MB
7 Execution time: 0.653790 seconds
Memory Usage: 10552.25 MB
8 Execution time: 1.551191 seconds
Memory Usage: 10552.51 MB
9 Execution time: 1.585286 seconds
Memory Usage: 10510.06 MB
10 Execution time: 0.885810 seconds
Memory Usage: 10516.25 MB
11 Execution time: 0.923901 seconds
Memory Usage: 10516.51 MB
12 Execution time: 2.430333 seconds
Memory Usage: 10522.18 MB
13 Execution time: 0.643348 seconds
Memory Usage: 10522.44 MB
14 Execution time: 0.663068 seconds
Memory Usage: 10522.44 MB
15 Execution time: 0.683088 seconds
Memory Usage: 10528.37 MB
16 Execution time: 1.745989 seconds
Memory Usage: 10528.37 MB
17 Execution time: 0.664083 seconds
Memory Usage: 10528.62 MB
18 Execution t

In [17]:
metadata['Clip_Predictions'] = predicted_cover_labels_col
metadata['Image_Valid'] = images_valid

In [18]:
colName = []
colValue = []
for i in range(int(tops.shape[1]/2)):
    colName.append(f"top{1+i}")
    colValue.append(f"top{1+i}value")
colTop = colName + colValue


# Create a DataFrame from the array
toppd = pd.DataFrame(tops, columns=colTop)

# Ensure both DataFrames have the same number of rows before concatenation
if metadata.shape[0] == toppd.shape[0]:
    metadata = pd.concat([metadata.reset_index(drop=True), toppd.reset_index(drop=True)], axis=1)
else:
    print("Error: metadata and top5pd have different row counts!")

# Save to Excel
if not os.path.exists(f"{save_dir}/NoThreshold/"):
    os.makedirs(f"{save_dir}/NoThreshold/")

output_path = f"{save_dir}/NoThreshold/{dn}_Predictions.csv"
metadata.to_csv(output_path, index=False)
#metadata.to_csv(os.path.join(save_dir,'PredResFL_02282025.csv'), index=False)

print(f"Predictions saved to: {output_path}")

Predictions saved to: CLIP-GPU-All_FLv4-04072025-part3/NoThreshold/ALL_FL_Predictions.csv


In [19]:
#nvidia-smi | grep ' C ' | awk '{print $5}' | xargs -I {} kill -9 {} #kill memory if outage

In [20]:
# plt.figure(figsize=(10, 20))
# for i in range(100, 106):
#     plt.subplot(6, 1, i-100 + 1)
#     plt.imshow(original_images[i])
#     top_5_preds = list(metadata.loc[i,['top1', 'top2', 'top3','top4', 'top5']])
#     top_5_preds_str = ', '.join(top_5_preds)
#     plt.title(f"Predict:{predicted_cover_labels[i]}\nTop5 Predict:{top_5_preds_str}")
#     plt.xticks([])
#     plt.yticks([])

In [21]:
'''
# Create a plot for each image with its actual and predicted class
plt.figure(figsize=(3, 3))
for i in range(len(original_images)):
    #print(i)
    #plt.subplot(len(original_images), 1, i + 1)
    plt.imshow(original_images[i])
    top_5_preds = list(metadata.loc[i,['top1', 'top2', 'top3','top4', 'top5']])
    top_5_preds_str = ', '.join(top_5_preds)
    plt.title(f"Predict:{predicted_cover_labels[i]}\nTop5 Predict:{top_5_preds_str}")
    plt.xticks([])
    plt.yticks([])

    # Save the image with renamed file
    plt.savefig(os.path.join(save_dir, f"{i}_{photo_ids[i]}_pred_{predicted_cover_labels[i]}_top5Pred_{top_5_preds_str}.png"), bbox_inches='tight')
    plt.close()  # Close the figure to avoid display overlap
'''

'\n# Create a plot for each image with its actual and predicted class\nplt.figure(figsize=(3, 3))\nfor i in range(len(original_images)):\n    #print(i)\n    #plt.subplot(len(original_images), 1, i + 1)\n    plt.imshow(original_images[i])\n    top_5_preds = list(metadata.loc[i,[\'top1\', \'top2\', \'top3\',\'top4\', \'top5\']])\n    top_5_preds_str = \', \'.join(top_5_preds)\n    plt.title(f"Predict:{predicted_cover_labels[i]}\nTop5 Predict:{top_5_preds_str}")\n    plt.xticks([])\n    plt.yticks([])\n\n    # Save the image with renamed file\n    plt.savefig(os.path.join(save_dir, f"{i}_{photo_ids[i]}_pred_{predicted_cover_labels[i]}_top5Pred_{top_5_preds_str}.png"), bbox_inches=\'tight\')\n    plt.close()  # Close the figure to avoid display overlap\n'